![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# TFM - Impacto de la alimentación sobre la salud de las personas

## Tabla de Contenidos
* [1. Introducción](#introduccion)
    * [1.1. Importar Librerias](#librerias)
    * [1.2. Importar datos](#data)
        * [1.2.1. Consumo de alimentos por CC.AA.](#consumos)
        * [1.2.2. Población por CC.AA.](#poblacion)
        * [1.2.3. Muertes por diabetes mellitus por CC.AA](#diabetes)


       
* [2. Extracción y limpieza de los datos](#etl)
    * [2.1. Selección de las familias de alimentos relacionadas con la diabetes](#familias_diabetes)
    * [2.2. Normalización de consumos de alimentos con la poblacion de las CC.AA.](#normalizacion_consumos)
    * [2.3. Transformación dataset de muertes por diabetes mellitus por CC.AA.](#etl_diabetes)


       
* [3. Análisis de los datos](#analisis)
    * [3.1. Clustering de CC.AA.](#clustering)

# 1. Introducción <a class="anchor" id="introduccion"></a>

## 1.1. Importar Librerías<a class="anchor" id="librerias"></a>

In [891]:
import os
import sys
import numpy as np
import pandas as pd
from datetime import datetime

from matplotlib import pyplot as plt
import plotly.express as px


import seaborn as sns
from sklearn import preprocessing

import pickle


## 1.2. Importar datos <a class="anchor" id="data"></a>

### 1.2.1. Consumos por CC.AA. <a class="anchor" id="consumos"></a>

In [892]:
PATH_COMUNIDADES = 'data/comunidades/'

In [893]:
# Leemos los nombres de los ficheros
paths_comunidades = [path for path in os.listdir(PATH_COMUNIDADES) if path.endswith('.xlsx')]

# Creamos una lista para ir añadiendo los dataframes leidos
dfs = list()

for path_comunidad in paths_comunidades:
    # Extraemos el nombre de la comunidad
    nombre_comunidad = os.path.splitext(path_comunidad)[0]

    # Leemos el fichero, reseteamos el index y le cambiamos el nombre de la columna a ID
    fichero = pd.read_excel('./data/comunidades/'+path_comunidad).reset_index(drop=True).rename(columns={'Unnamed: 0': 'ID'})

    # Descartamos las filas donde el la columna BIO no es nula y asignamos el nombre de la comunidad a una nueva columna.
    fichero_sin_bio = fichero[fichero['SEGMENTO BIO'].isna()].reset_index(drop=True).assign(COMUNIDAD=nombre_comunidad)

    # Añadimos el fichero leido a nuestra lista de dataframes
    dfs.append(fichero_sin_bio)

# Montamos el dataframe final
df_comunidades = pd.concat(dfs, ignore_index=True)

# Eliminamos la columna SEGMENTO BIO
df_comunidades.drop('SEGMENTO BIO', axis=1, inplace=True)

# Cambiamos el orden de la columna COMUNIDAD
cols = df_comunidades.columns.tolist()
cols = cols[:5] + cols[-1:] + cols[5:-1]
df_comunidades = df_comunidades[cols]

In [894]:
df_comunidades

,ID,SECTOR,SECCION,CATEGORIA,FAMILIA,COMUNIDAD,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,Ceuta_y_Melilla,0.00,0.00,0.00,0.00,...,2652.71,2853.52,2699.68,2703.08,2484.19,2853.67,2844.06,2412.15,2559.18,2647.53
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,Ceuta_y_Melilla,0.00,0.00,0.00,0.00,...,1328.36,1421.87,1364.16,1405.32,1269.16,1457.10,1477.37,1288.87,1375.99,1442.49
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,Ceuta_y_Melilla,0.00,0.00,0.00,0.00,...,704.00,782.74,760.74,791.09,717.49,829.03,829.07,739.01,813.17,891.25
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,Ceuta_y_Melilla,0.00,0.00,0.00,0.00,...,60.80,72.97,67.83,71.55,62.32,73.07,76.68,61.45,60.28,63.69
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,Ceuta_y_Melilla,0.00,0.00,0.00,0.00,...,18.18,20.78,20.41,20.16,18.68,21.06,22.47,16.94,17.49,19.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9427,REQUESON,PROD.FRESCOS,QUESOS,REQUESON,NaN,Canarias,0.40,0.67,0.83,0.80,...,9.39,9.57,9.29,9.08,9.70,10.47,10.78,9.98,10.36,10.56
9428,Q.REQUESON,PROD.FRESCOS,QUESOS,REQUESON,Q.REQUESON,Canarias,0.40,0.67,0.83,0.80,...,9.39,9.57,9.29,9.08,9.70,10.47,10.78,9.98,10.36,10.56
9429,RESTO QUESOS NATURALES,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,NaN,Canarias,21.54,23.89,24.21,26.09,...,95.81,98.23,106.96,97.40,96.91,104.94,114.84,110.55,110.09,112.98
9430,RQN.AL CORTE,PROD.FRESCOS,QUESOS,RESTO QUESOS NATURALES,RQN.AL CORTE,Canarias,2.51,1.90,3.12,3.19,...,30.57,33.69,34.43,32.22,32.51,34.06,35.54,32.48,36.00,36.14


### 1.2.2. Población por CC.AA. <a class="anchor" id="poblacion"></a>

In [895]:
PATH_POBLACION = './data/poblacion/'

In [896]:
df_poblacion = pd.read_excel(PATH_POBLACION+'poblacion_comunidades.xlsx', skiprows=1).set_index('CC.AA.')

In [897]:
df_poblacion

,2016,2017,2018,2019,2020
CC.AA.,,,,,
Ceuta_y_Melilla,170545,171079,171528,171264,171278
La_Rioja,315794,315381,315675,316798,313914
Pais_Vasco,2189534,2194158,2199088,2207776,2220504
Navarra,640647,643234,647554,654214,661197
Murcia,1464847,1470273,1478509,1493898,1511251
Madrid,6466996,6507184,6578079,6663394,6779888
Galicia,2718525,2708339,2701743,2699499,2701819
Extremadura,1087778,1079920,1072863,1067710,1063987
C_Valenciana,4959968,4941509,4963703,5003769,5057353


###  1.2.3. Muertes por diabetes mellitus por CC.AA <a class="anchor" id="diabetes"></a>

In [898]:
# Guardamos el path donde tenemos los datos de las enfermedades
PATH_ENFERMEDADES = './data/enfermedades/'

In [899]:
# Creamos el DataFrame con los datos de la tasa de muertes por comunidades
df_muertos_diabetes = pd.read_excel(PATH_ENFERMEDADES + 'diabetes.xlsx', skiprows=1)

In [900]:
# Comprobamos que se ha cargado correctamente el DataFrame
df_muertos_diabetes

,CC.AA.,2016,2017,2018,2019,2020
0,Ceuta_y_Melilla,50.36,35.33,27.14,20.33,47.11
1,La_Rioja,15.88,15.51,19.18,15.20,15.82
2,Pais_Vasco,15.19,13.03,15.14,15.07,16.65
3,Navarra,20.41,20.21,16.13,15.77,16.34
4,Murcia,20.92,25.17,21.24,22.40,30.91
5,Madrid,10.90,10.49,10.23,8.28,10.03
6,Galicia,16.32,14.40,12.66,11.08,12.88
7,Extremadura,19.58,17.68,17.88,16.88,22.17
8,C_Valenciana,19.39,20.06,25.27,25.50,28.49
9,Catalunya,19.32,20.81,19.91,18.47,21.28


# 2. Extracción y limpieza de los datos <a class="anchor" id="etl"></a>

## 2.1. Selección de las familias de alimentos relacionadas con la diabetes <a class="anchor" id="familias_diabetes"></a>

Nuestros datos están clasificados de la siguiente forma:

SECTOR > SECCION > CATEGORIA > FAMILIA

El objetivo es ir recorriendo cada agrupación para descartar familias de alimentos que no estén relacionadas con la diabetes.

In [901]:
# Sacamos la lista de todos los valores de cada sector, sección, categoría y familia de alimentos
lista_sectores = list(df_comunidades[~df_comunidades['SECTOR'].isna()]['SECTOR'].unique())
lista_secciones = list(df_comunidades[~df_comunidades['SECCION'].isna()]['SECCION'].unique())
lista_categorias = list(df_comunidades[~df_comunidades['CATEGORIA'].isna()]['CATEGORIA'].unique())
lista_familias = list(df_comunidades[~df_comunidades['FAMILIA'].isna()]['FAMILIA'].unique())

### Sectores

In [902]:
# Copiamos la primera lista para modificarla
lista_sectores_diabetes = lista_sectores.copy()

# Eliminamos los sectores que no están relacionados con la diabetes
lista_sectores_diabetes.remove('DROGUERIA Y PERFUMERIA')

# Mostramos los sectores seleccionados
lista_sectores_diabetes

['ALIM. Y BEBIDAS', 'PROD.FRESCOS']

### Secciones

In [903]:
# Seleccionamos las secciones que hay dentro de de los sectores seleccionados en lista_sectores_diabetes
lista_secciones_sectores_diabetes = list(df_comunidades.loc[df_comunidades['SECTOR'].isin(lista_sectores_diabetes), 'SECCION'].unique())[1:]

# Copiamos la lista de los secciones de los sectores relacionados con la diabetes
lista_secciones_diabetes = lista_secciones_sectores_diabetes.copy()

# Creamos una lista de las secciones a eliminar
secciones_sin_diabetes = ['CONSERVAS', 'CHARCUTERIA', 'PL. COCINADOS Y PRECOCINADOS', 'QUESOS']

# Eliminamos las secciones que no están relacionadas con la diabetes
for i in range(len(secciones_sin_diabetes)):
    lista_secciones_diabetes.remove(secciones_sin_diabetes[i])

# Mostramos las secciones seleccionadas
lista_secciones_diabetes

['ALIM.SECA', 'BEBIDAS', 'LECHE Y BATIDOS', 'CONGELADOS', 'DERV.LACTEOS']

### Categorías

In [904]:
# Seleccionamos las categorias que hay dentro de de los secciones seleccionados en lista_secciones_diabetes
lista_categorias_secciones_diabetes = list(df_comunidades.loc[df_comunidades['SECCION'].isin(lista_secciones_diabetes), 'CATEGORIA'].unique())[1:]

# Copiamos la lista de los categorias de las secciones relacionados con la diabetes
lista_categorias_diabetes = lista_categorias_secciones_diabetes.copy()

# Creamos una lista de las categorias a eliminar
categorias_sin_diabetes = [
    'ACEITE', 'ADITIVOS COCINA', 'ALIMENTOS INFANTILES', 'ALIMENTOS MASCOTAS', 'APERITIVOS', 'ARROZ', 'CAFES', 'DIETETICOS', 'FRUTOS SECOS',
    'INFUSIONES', 'LEGUMBRES SECAS', 'PANADERIA INDUSTRIAL', 'PASTAS', 'SOPAS Y DESHIDRATADOS', 'AGUAS', 'CERVEZA', 'ESPUMOSOS', 'VINOS',
    'LECHE LIQUIDA Y BEBIDAS VEGETALES', 'PESCADO CONGELADO', 'PESCADO PREPARADO CONGELADO', 'PLATOS PREPARADOS CONGELADOS', 'VERDURAS CONGELADAS',
    'MANTEQUILLA', 'MARGARINA', 'QUESO TIPO PETIT'
]

# Eliminamos las categorias que no están relacionadas con la diabetes
for i in range(len(categorias_sin_diabetes)):
    lista_categorias_diabetes.remove(categorias_sin_diabetes[i])

# Mostramos las categorías seleccionadas
lista_categorias_diabetes

['AZUCAR Y EDULCORANTES',
 'BOLLERIA INDUSTRIAL',
 'CACAO',
 'CEREALES DESAYUNO',
 'CHOCOLATES',
 'DULCES NAVIDENOS',
 'GALLETAS',
 'GOLOSINAS',
 'REPOSTERIA',
 'SALSAS',
 'BEBIDAS ALCOHOLICAS',
 'BEBIDAS REFRESCANTES',
 'ZUMOS',
 'BATIDOS',
 'HORCHATA.',
 'LECHE NO LIQUIDA',
 'HELADOS',
 'OTROS CONGELADOS',
 'NATA',
 'POSTRES PREPARADOS',
 'YOGURES']

In [905]:
# Seleccionamos las familias de alimentos que hay dentro de de las categorias seleccionados en lista_categorias_diabetes
lista_familias_categorias_diabetes = list(df_comunidades.loc[df_comunidades['CATEGORIA'].isin(lista_categorias_diabetes), 'FAMILIA'].unique())[1:]

# Copiamos la lista de las familias de alimentos de las categorias relacionados con la diabetes
lista_familias_diabetes = lista_familias_categorias_diabetes.copy()

# Creamos una lista de las categorias a eliminar
famlias_sin_diabetes = [
    'EDULCORANTE', 'LEVADURA', 'L.EVAPORADA', 'L.POLVO', 'BASES PIZZA', 'CHURROS', 'FRUTAS', 'HIELO', 'HOJALDRES MASAS RELLENAS', 'HOJALDRES SIN RELLENAR',
    'NATA', 'YOGUR NATURAL'
]

# Eliminamos las familias que no están relacionadas con la diabetes
for i in range(len(famlias_sin_diabetes)):
    lista_familias_diabetes.remove(famlias_sin_diabetes[i])

# Lista de las categorias relacionadas con la diabetes. Mostramos la cantidad de familias de alimentos que hemos seleccionado
len(lista_familias_diabetes)

120

## 2.2. Normalización de consumos de alimentos con la poblacion de las CC.AA. <a class="anchor" id="normalizacion_consumos"></a>

In [906]:
# Creamos un dataset de las familias de alimentos que tienen relación con la diabetes
df_diabetes = df_comunidades[df_comunidades['FAMILIA'].isin(lista_familias_diabetes)].groupby('COMUNIDAD').sum().T

# Editamos el índice para que tenga un formato adecuado para ser formateado a fecha (quitamos 'SEMANA', 1 para que la semana empiece en lunes, la semana y el año)
df_diabetes.index = '1 ' + df_diabetes.index.str[-5:-3] + ' ' + df_diabetes.index.str[-2:]
df_diabetes.index = pd.to_datetime(df_diabetes.index, format="%w %W %y")

# Cambiamos la granularidad a mensual
df_diabetes = df_diabetes.groupby(pd.Grouper(freq='M')).sum()

In [907]:
# Dividimos el DF para cada año
df_diabetes_2017 = df_diabetes['2017':'2017']
df_diabetes_2018 = df_diabetes['2018':'2018']
df_diabetes_2019 = df_diabetes['2019':'2019']

In [908]:
# Usamos la función apply para dividir los datos por la población de cada comunidad. Multiplicamos por 1000 para que nos den mejores valores
df_diabetes_2017_normalizado = df_diabetes_2017.apply(lambda x: x / df_poblacion['2017'][x.name] * 1000)
df_diabetes_2018_normalizado = df_diabetes_2018.apply(lambda x: x / df_poblacion['2018'][x.name] * 1000)
df_diabetes_2019_normalizado = df_diabetes_2019.apply(lambda x: x / df_poblacion['2019'][x.name] * 1000)

In [909]:
# Unimos el DF con los valores ya normalizados
df_diabetes_normalizado = pd.concat([df_diabetes_2017_normalizado, df_diabetes_2018_normalizado, df_diabetes_2019_normalizado])

In [910]:
fig = px.line(
    df_diabetes_normalizado,
    title='Consumo normalizado de alimentos relacionados con la diabetes por CCAA',
    labels = {
        'index' : 'Tiempo',
        'value' : 'Consumo / Población * 100'
    }
)
fig.show()

## 2.3. Transformación dataset de muertes por diabetes mellitus por CC.AA. <a class="anchor" id="etl_diabetes"></a>

Los datos de diabetes mellitus que tenemos se refieren a la *** Tasa de mortalidad ajustada por diabetes mellitus, por 100 000 hab. ***

Por lo tanto para obtener los datos aproximados de muertes tenemos que dividir la población de cada CC.AA entre 100.000 y multiplicar por la tasa.

El objetivo de este punto es transformar el dataset para que tenga una granularidad similar a la de los datos de consumos.

In [911]:
# Eliminamos el año 2016
df_muertos_diabetes.drop(['2016'], axis=1, inplace=True)

# Establecemos la CCAA como indice y transponemos la tabla
df_muertos_diabetes = df_muertos_diabetes.set_index('CC.AA.').T

# Cambiamos el index a Datetime
df_muertos_diabetes.index = pd.to_datetime(df_muertos_diabetes.index, format="%Y")

# Cambiamos la granularida a mensual con un backfill
df_muertos_diabetes = df_muertos_diabetes.resample('M').bfill()

# Eliminamos las 3 ultimas filas para que el último dato sea de Octubre de 2019, igual que los datos de consumos
df_muertos_diabetes.drop(df_muertos_diabetes.tail(3).index , axis=0, inplace=True)

# Mostramos el resultado de nuestro dataset
df_muertos_diabetes.head()

CC.AA.,Ceuta_y_Melilla,La_Rioja,Pais_Vasco,Navarra,Murcia,Madrid,Galicia,Extremadura,C_Valenciana,Catalunya,Castilla_La_Mancha,Castilla_y_Leon,Cantabria,Canarias,Baleares,Asturias,Aragon,Andalucia
2017-01-31,27.14,19.18,15.14,16.13,21.24,10.23,12.66,17.88,25.27,19.91,18.72,19.46,13.06,23.11,24.07,19.93,20.84,20.95
2017-02-28,27.14,19.18,15.14,16.13,21.24,10.23,12.66,17.88,25.27,19.91,18.72,19.46,13.06,23.11,24.07,19.93,20.84,20.95
2017-03-31,27.14,19.18,15.14,16.13,21.24,10.23,12.66,17.88,25.27,19.91,18.72,19.46,13.06,23.11,24.07,19.93,20.84,20.95
2017-04-30,27.14,19.18,15.14,16.13,21.24,10.23,12.66,17.88,25.27,19.91,18.72,19.46,13.06,23.11,24.07,19.93,20.84,20.95
2017-05-31,27.14,19.18,15.14,16.13,21.24,10.23,12.66,17.88,25.27,19.91,18.72,19.46,13.06,23.11,24.07,19.93,20.84,20.95


In [912]:
fig = px.line(
    df_muertos_diabetes,
    title='Tasa ajustada de muertes por diabetes mellitus por CC.AA.',
    labels = {
        'index' : 'Tiempo',
        'value' : 'Tasa de muertes (*Poblacion / 100.000'
    }
)
fig.show()